In [349]:
import numpy as np

In [350]:
def printM3(matrix3, text):
    print(f"\n{text}:\n[")
    for i in range(len(matrix3)):
        print(matrix3[i])
    print("]")

In [351]:
def calcFoward(values, weights):
    #print(f"values:{values}\nweights:{weights}")
    result = 0.0
    for i in range(len(weights)-1):
        result += values[i] * weights[i]
    result += 1 * weights[-1]
    #print(f"\ncalcFoward:{result}")
    return result

def calcCost(value, ideal):
    result = (ideal - value)**2/2
    #print(f"Cost:{result}")
    return result

def calcdETotalGo(value, ideal):
    result = -(ideal - value)
    #print(f"dETotal:{result}")
    return result

In [352]:
def relu(value):
    if(value < 0):
        value = 0
    print(f"\relu:{value}")
    return value

def drelu(value):
    if(value > 0):
        return value
    else:
        return 0

def linear(value):
    return value

def dlinear(value):
    return 1

def sigmoid(value):
    try:
        value = (1/(1 + np.exp(-value)))
    except OverflowError:
        value = np.finfo(np.float64).tiny
    return value

def dsigmoid(value):
    value = value * (1 - value)
    return value

In [353]:
class Layer:
    def __init__(self,layer_size, neuron_size, activation_function, dactivation_function, weights):
        #self.neurons = np.random.rand(layer_size, neuron_size+1)
        matrix = np.array(weights)
        self.neurons = matrix
        print(f"\nNEURONS:{self.neurons}")
        self.activation_function = activation_function
        self.dactivation_function = dactivation_function
    
    def updateWeights(self, update_values, alpha):
        #print(f"Udate Weights\nUpdate Values:{update_values}\nAlpha:{alpha}")
        for i in range(len(self.neurons)):
            #print(f"\nNeuron {i}")
            for j in range(len(self.neurons[i])):
                #print(f"\nWeight {j}")
                #print(f"\nValue:{update_values[i][j]}")
                #print(f"Equation\n{self.neurons[i][j]} - {alpha} * {update_values[i][j]}")
                self.neurons[i][j] = self.neurons[i][j] - alpha * update_values[i][j]


    def foward(self,values):
        layer_a = []
        layer_z = []
        for i in range(len(self.neurons)):
            #print(f"\nNeuron:{i}")
            z = calcFoward(values,self.neurons[i])
            a = self.activation_function(z)
            #print(f"a:{a}\nz:{z}")
            layer_a.append(a)
            layer_z.append(z)
        #print(f"\nLayer A:{layer_a}\nLayer Z:{layer_z}")
        return np.array(layer_a),np.array(layer_z)
    
    def backpropagation(self, a0, a1, z0, ideals):
        #print(f"A0:{a0}\nA1:{a1}\nZ:{z0}\nIdeals:{ideals}")
        result = []
        aux = []
        for i in range(len(self.neurons)):
            neuron = []
            #print(f"\nNeuron:{i}")
            dEg = calcdETotalGo(a0[i],ideals[i])
            #print(f"dEg:{dEg}")
            dgouo = self.dactivation_function(a0[i])
            #print(f"dgouo:{dgouo}")
            #pesos
            for j in range(len(self.neurons[i])-1):
                #print(f"\nWeight:{j}")
                dgh = a1[j]
                #print(f"dgh:{dgh}")
                dEw = dEg * dgouo * dgh
                #print(f"dEw:{dEw}")
                neuron.append(dEw)
            #bias
            #print("\nBias:")
            dEw = dEg * dgouo
            #print(f"dEw:{dEw}")
            neuron.append(dEw)
            #print(f"Neuron Result:{neuron}")
            result.append(np.array(neuron))
            aux.append(dEw)
        return np.array(result), np.array(aux)

    def backpropagationHidden(self, bias, a0, a1, next_layer_size):
        #print(f"\nA0:{a0}\nA1:{a1}\nBias2:{bias}\nWeights:{self.neurons}\nNext Layer Size:{next_layer_size}")
        result = []
        for i in range(len(self.neurons)):
            neuron = []
            #print(f"\nNeuron:{i}")
            dghuh = self.dactivation_function(a0[i])
            #print(f"dghuh:{dghuh}")
            true_dEg = []
            for j in range(next_layer_size):
                #print(f"\nCalc dEw:{j}")
                dEg = self.neurons[j][i] * bias[j] 
                #print(f"dEg:{dEg}")
                #print(f"W:{self.neurons[j][i]}\nB:{bias[j]}\ndEg:{dEg}")
                true_dEg.append(dEg)
            true_dEg = sum(true_dEg)
            #print(f"\nTrue dEg:{true_dEg}")
            #pesos
            for k in range(len(self.neurons[i])-1):
                #print(f"\nWeight:{k}")
                dgh = a1[k]
                #print(f"dgh:{dgh}")
                dEw = true_dEg * dghuh * dgh
                #print(f"\ndEw:{dEw}")
                neuron.append(dEw)
            #bias
            true_dEg = []
            for j in range(next_layer_size):
                #print(f"\nCalc dEw:{j}")
                dEg = self.neurons[j][i] * bias[j] 
                #print(f"\nB:{bias[j]}\ndEg:{dEg}")
                true_dEg.append(dEg)
            true_dEg = sum(true_dEg)
            #print(f"\nTrue dEg:{true_dEg}")
            dEw = true_dEg * dghuh
            #print(f"\ndEw:{dEw}")
            neuron.append(dEw)
            result.append(np.array(neuron))


        return np.array(result)


    def relat(self):
        print(self.neurons)

In [354]:
class Network:
    def __init__(self):
        self.layers = []

    def appendLayer(self, layer_size, neuron_size, activation_function, dactivation_function, weights):
        self.layers.append(Layer(layer_size, neuron_size, activation_function, dactivation_function, weights))

    def foward(self,values):
        #print(f"\nFirst_Values:{values}")
        layer_a = [values]
        layer_z = []
        for i in range(len(self.layers)):
            #print(f"\nFoward:{i}")
            values,z = self.layers[i].foward(values)
            layer_a.append(values)
            layer_z.append(z)
        layer_a = np.array(layer_a)
        layer_z = np.array(layer_z)
        return layer_a,layer_z
                
    def backpropagation(self, all_a, all_z, ideals):
        #print(f"\nBack First")
        #printM3(all_a,"All A")
        #printM3(all_z,"All Z")
        #print(f"\nIdeals:{ideals}")
        result = []
        next = []
        for i in range(len(self.layers)):
            if i == 0:
                #print(f"\nBack")
                aux, next = self.layers[i].backpropagation(all_a[-(i+1)], all_a[-(i+2)], all_z[-(i+1)], ideals)
                result.append(aux)
                #print(next)
            else:
                #print(f"back hidden{i}")
                aux = self.layers[i].backpropagationHidden(next, all_a[-(i+1)], all_a[-(i+2)], len(self.layers[i-1].neurons))
                result.append(aux)
        return result
        
    def calcError(self, output, ideals):
        result = 0
        #print(f"Ideals:{ideals}\nOut:{output}")
        for i in range(len(output)):
            result = result + calcCost(output[i], ideals[i])
        #print(f"Total cost:{result}")
        return result
    
    def updateWeights(self, update_values, alpha):
        for i in range(len(self.layers)):
            self.layers[-(i+1)].updateWeights(update_values[i],alpha)

    def train(self, input, ideal, alpha, epoch):
        acumulated = 0
        for i in range(epoch):
            error = []
            for j in range(len(input)):
                all_a, all_z = self.foward(input[j])
                if(i <= 2 or i >= 998):
                    error.append(self.calcError(all_a[-1],ideal[j]))
                fix = self.backpropagation(all_a, all_z, ideal[j])
                #print(f"fix:{fix}")
                if type(acumulated) is int:
                    acumulated = np.array(fix)
                else:
                    acumulated = acumulated + np.array(fix)
            if(i <= 2 or i >= 998):
                error = sum(error)
                error = error / len(input)
                print(f"Total Error:{error}")
            #print(f"Acumulated sum:{acumulated}")
            acumulated = acumulated / len(input)
            #print(f"Acumulated div:{acumulated}")
            self.updateWeights(acumulated,alpha)

    
    def relat(self):
        for i in range(len(self.layers)):
            print(f"\nLayer{i}:")
            self.layers[i].relat()


In [355]:
network = Network()
network.appendLayer(2,2,sigmoid,dsigmoid,[[0.15, 0.2, 0.35], [0.25, 0.3, 0.35]])
network.appendLayer(2,2,sigmoid,dsigmoid,[[0.4, 0.45, 0.6], [0.5, 0.55, 0.6]])

#network.relat()

# C1 C2 R
# 1  1  0
# 0  1  1
# 1  0  1
# 0  0  0

input = [[1,1],[0.05,0.1],[1,0],[0,0]]
all_ideals = [[0,1],[0.01,0.99],[1,0],[0,1]]

#input = [[0.05,0.1]]
#all_ideals = [[0.01,0.99]]

clean_input = []
for i in range(len(input)):
    clean_input.append(np.array(input[i]))

#print(clean_input)

clean_all_ideals = np.array(all_ideals)


#all_a, all_z = network.foward(clean_input[1])

#printM3(all_a,"All A")
#printM3(all_z,"All Z")

#fix = network.backpropagation(all_a, all_z, clean_all_ideals[1])

#print(f"\nFix:{fix}")

#network.updateWeights(fix,0.5)

network.train(clean_input, clean_all_ideals, 0.5, 1001)

network.relat()


NEURONS:[[0.15 0.2  0.35]
 [0.25 0.3  0.35]]

NEURONS:[[0.4  0.45 0.6 ]
 [0.5  0.55 0.6 ]]
Total Error:0.31375990341439
Total Error:0.3059056742091283
Total Error:0.2959755474353671
Total Error:0.010192038201983203
Total Error:0.010166977893006022
Total Error:0.010142022838106902

Layer0:
[[-3.33826444  3.28731329  1.13587435]
 [-3.17964362  3.19361613  1.02272674]]

Layer1:
[[-3.32076989 -3.09065874  2.27707832]
 [ 3.31856025  3.17282679 -2.3125309 ]]
